# Separando o dataset com temporal_train_test

In [879]:
# from sktime.forecasting.model_selection import temporal_train_test_split

In [880]:
# seller_id = 100000060
# time_serie = model_df[model_df['Seller ID'] == seller_id]
# time_serie.drop(columns=['Seller ID','Day of Week','Last Week','Last Month'],inplace=True)
# time_serie['Change'] = time_serie['Change'].astype('Int64')
# time_serie

# Lag de um dia na nossa time serie

In [881]:
# time_serie.set_index('Date',inplace=True)

In [882]:
# target = time_serie.pop('Transactioned')
# y = pd.DataFrame(data=target,columns=['Transactioned'])
# y

In [883]:
# time_serie = time_serie.shift(1)
# time_serie.drop(pd.to_datetime('2024-01-30'), inplace=True)
# time_serie

In [884]:
# date_to_pred = pd.to_datetime('2024-11-04')
# time_serie = time_serie[time_serie.index < date_to_pred]
# y = y[y.index<date_to_pred]
# y.drop(pd.to_datetime('2024-01-30'), inplace=True)
# y

In [885]:
# time_serie

In [886]:
# y_train, y_test, X_train, X_test = temporal_train_test_split(y,time_serie,test_size=20)

# Creating forecasting

In [887]:
# from sktime.forecasting.base import ForecastingHorizon
# import numpy as np

# fh = ForecastingHorizon(np.arange(1,len(y_test)+2),is_relative=True)

In [888]:
# from xgboost import XGBRegressor

# # Calculando scale_pos_weight
# n_pos = len(y_train[y_train['Transactioned'] == 1])
# n_neg = len(y_train[y_train['Transactioned'] == 0])
# scale_pos_weight = n_neg / n_pos

# xgb_model = XGBRegressor(
#     n_estimators=100,
#     learning_rate=0.1,
#     max_depth=5,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     random_state=42,
#     scale_pos_weight=scale_pos_weight
# )

In [889]:
# from sktime.forecasting.compose import make_reduction

# forecaster = make_reduction(xgb_model, window_length=12, strategy="recursive")

In [890]:
# X_train.info()

In [891]:
# forecaster.fit(y=y_train,X=X_train)

In [892]:
# y_pred = forecaster.predict(fh=fh,X=X_test)
# y_pred

In [893]:
# from sktime.utils.plotting import plot_series
# plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])

In [894]:
# # Análise de erros
# errors = abs(y_test['Transactioned'].values - y_pred.values)
# print("Erro Médio Absoluto (MAE):", errors.mean())

# Iterando em sellers

In [895]:
import pandas as pd
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon
from xgboost import XGBRegressor
from sktime.forecasting.compose import make_reduction
from sktime.utils.plotting import plot_series
import numpy as np
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sktime.forecasting.model_selection import SlidingWindowSplitter,ForecastingGridSearchCV, ForecastingOptunaSearchCV
from sktime.split import SingleWindowSplitter
from tqdm import tqdm
from sktime.performance_metrics.forecasting import MeanAbsoluteError
from optuna import Trial
import optuna
from concurrent.futures import ProcessPoolExecutor
import warnings
from sklearn.model_selection import GridSearchCV
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# Reading DF

In [896]:
path = 'C:/workspace/Desafio Grão Direto IA/Data/model_df.xlsx'
model_df = pd.read_excel(path)
model_df = model_df.drop(columns=[ 'price_percentage_change', 'smap', 'emap', 'cmap',
                          'cbot_percentage_change', 'smacb', 'emacb', 'cmacb',
                         'smad', 'emad', 'cmad'])
model_df

,Date,Seller ID,Last Transaction,Last Week,Last Month,Day of Week,Transactioned,Price,price_change,CBOT,cbot_change,Close,Change dolar,Percentage Change dolar
0,2024-01-30,100000060,0,0,0,1,0,97.111044,0,1215.886963,0,4.950200,0,0.000000
1,2024-01-31,100000060,0,0,0,2,0,98.585604,1,1222.299024,1,4.950300,1,0.002023
2,2024-02-01,100000060,0,0,0,3,0,96.726709,0,1209.233105,0,4.951900,1,0.032307
3,2024-02-02,100000060,0,0,0,4,0,97.929889,1,1199.996115,0,4.915000,0,-0.750764
4,2024-02-03,100000060,0,0,0,5,0,97.929889,0,1199.996115,0,4.915000,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204001,2024-11-01,100000832,240,0,0,4,0,117.601042,1,1012.899216,0,5.789400,1,0.468098
204002,2024-11-02,100000832,241,0,0,5,0,117.601042,0,1012.899216,0,5.789400,0,0.000000
204003,2024-11-03,100000832,242,0,0,6,0,117.601042,0,1012.899216,0,5.789400,0,0.000000
204004,2024-11-04,100000832,243,0,0,0,0,115.506245,0,1020.956401,1,5.677526,0,-1.970473


# Modelos com hyperparameter tuning

## GridSearch

In [897]:
# def process_seller_gs(seller_id):
#     #print(f"Processando Seller ID: {seller_id}")
    
#     # Filtrar os dados para o Seller ID atual
#     time_serie = model_df[model_df['Seller ID'] == seller_id]
#     time_serie = time_serie.drop(columns=['Seller ID','Day of Week'])
    
#     # Preparação da série temporal
#     time_serie.set_index('Date', inplace=True)
#     target = time_serie.pop('Transactioned')
#     y = pd.DataFrame(data=target, columns=['Transactioned'])
#     time_serie = time_serie.shift(1)
    
#     # Remover valores fora do intervalo de datas desejado
#     date_to_pred = pd.to_datetime('2024-11-04')
#     first_date = pd.to_datetime('2024-01-30')
#     mask = (time_serie.index > first_date) & (time_serie.index < date_to_pred)
#     time_serie = time_serie.loc[mask]
#     y = y.loc[mask]

#     # Dividir os dados em treino e teste
#     y_train, y_test, X_train, X_test = temporal_train_test_split(y, time_serie, test_size=2)
#     scaler = StandardScaler()
#     X_train_scaled = scaler.fit_transform(X_train)
#     X_test_scaled = scaler.transform(X_test)
#     X_train = pd.DataFrame(data=X_train_scaled, columns=X_train.columns, index=X_train.index)
#     X_test = pd.DataFrame(data=X_test_scaled, columns=X_test.columns, index=X_test.index)
    
#     # Configuração do Forecasting Horizon
#     fh = ForecastingHorizon([3], is_relative=True)
    
#     # Calculando scale_pos_weight
#     n_pos = len(y_train[y_train['Transactioned'] == 1])
#     n_neg = len(y_train[y_train['Transactioned'] == 0])
#     scale_pos_weight = n_neg / n_pos if n_pos > 0 else 1
    
#     # Configuração do modelo e busca
#     param_grid = {
#         'n_estimators': [100, 500],
#         'learning_rate': [0.01, 0.1],
#         'max_depth': [3, 6],
#         'subsample': [0.1, 0.3],
#         'colsample_bytree': [0.6, 0.8]
#     }

#     xgb_model = XGBRegressor(
#         objective='reg:squarederror',
#         random_state=42,
#         scale_pos_weight=scale_pos_weight
#     )
    
#     # Criar o modelo
#     forecaster = make_reduction(xgb_model, window_length=12, strategy="recursive")

#     cv = SingleWindowSplitter(window_length=len(y)-2,fh=2)

#     # Criar o ForecastingGridSearchCV
#     grid_search = ForecastingGridSearchCV(
#         forecaster=forecaster,
#         param_grid=param_grid,
#         cv=cv,
#         scoring=MeanAbsoluteError()
#     )

#     #Treinando modelo
#     grid_search.fit(y=y_train, X=X_train)
    
#     # Previsão
#     y_pred = grid_search.predict(fh=fh, X=X_test)
#     # y_pred_04 = y_pred.loc['2024-11-04']

#     # # Análise de erros
#     # errors = abs(y_test['Transactioned'].values - y_pred.values)
#     # mae = errors.mean()
#     # print(f"Erro Médio Absoluto (MAE) para Seller ID {seller_id}: {mae}")
    
#     # # Plot das séries
#     # plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])
#     # plt.title(f"Previsão para Seller ID: {seller_id}")
#     # plt.show()
    
#     # Armazenar os resultados

#     gs = grid_search.best_params_

#     seller = {
#         "Seller ID": seller_id,
#         "Date": y_pred.index.values[0],  # Extract the single date value
#         "Score": y_pred['Transactioned'].values[0],  # Extract the single score value
#         "colsample_bytree": gs['colsample_bytree'],
#         "learning_rate": gs['learning_rate'],
#         "max_depth": gs['max_depth'],
#         "n_estimators": gs['n_estimators'],
#         "subsample": gs['subsample']
#     }

#     return seller

# # Lista de Seller IDs
# seller_ids = model_df['Seller ID'].unique()

# # Resultados para armazenar os erros por vendedor
# results = []

# # Criar a barra de progresso manualmente
# with ThreadPoolExecutor() as executor:
#     # Usar tqdm para iterar nos vendedores enquanto processa
#     for result in tqdm(executor.map(process_seller_gs, seller_ids), total=10, desc="Processando Sellers"):
#         results.append(result)

# # Criar DataFrame com os resultados
# results_df = pd.DataFrame(results)
# # results_df

72 minutos e 33,5 segundos - todos os Sellers

## OPTUNA

In [898]:
# import warnings
# warnings.filterwarnings("ignore", category=UserWarning, module='sktime')
# def process_seller_optuna(seller_id):
#     #print(f"Processando Seller ID: {seller_id}")
    
#     # Filtrar os dados para o Seller ID atual
#     time_serie = model_df[model_df['Seller ID'] == seller_id]
#     time_serie = time_serie.drop(columns=['Seller ID','Day of Week'])
    
#     # Preparação da série temporal
#     time_serie.set_index('Date', inplace=True)
#     target = time_serie.pop('Transactioned')
#     y = pd.DataFrame(data=target, columns=['Transactioned'])
#     time_serie = time_serie.shift(1)
    
#     # Remover valores fora do intervalo de datas desejado
#     date_to_pred = pd.to_datetime('2024-11-05')
#     first_date = pd.to_datetime('2024-01-30')
#     mask = (time_serie.index > first_date) & (time_serie.index < date_to_pred)
#     time_serie = time_serie.loc[mask]
#     y = y.loc[mask]

#     # Dividir os dados em treino e teste
#     y_train, y_test, X_train, X_test = temporal_train_test_split(y, time_serie, test_size=1)
#     scaler = StandardScaler()
#     X_train_scaled = scaler.fit_transform(X_train)
#     X_test_scaled = scaler.transform(X_test)
#     X_train = pd.DataFrame(data=X_train_scaled, columns=X_train.columns, index=X_train.index)
#     X_test = pd.DataFrame(data=X_test_scaled, columns=X_test.columns, index=X_test.index)
    
#     # Configuração do Forecasting Horizon
#     fh = ForecastingHorizon([1], is_relative=True)
    
#     # Calculando scale_pos_weight
#     n_pos = len(y_train[y_train['Transactioned'] == 1])
#     n_neg = len(y_train[y_train['Transactioned'] == 0])
#     scale_pos_weight = n_neg / n_pos if n_pos > 0 else 1
    
#     # Configuração do modelo e busca
#     param_grid = {
#         'n_estimators': optuna.distributions.IntDistribution(100, 1000),
#         'learning_rate': optuna.distributions.FloatDistribution(0.01, 0.8),
#         'max_depth': optuna.distributions.IntDistribution(2, 10),
#         'subsample': optuna.distributions.FloatDistribution(0.1, 0.8),
#         'colsample_bytree': optuna.distributions.FloatDistribution(0.2, 0.8)
#     }

#     xgb_model = XGBRegressor(
#         scale_pos_weight=scale_pos_weight
#     )
    
#     # Criar o modelo
#     forecaster = make_reduction(xgb_model, window_length=12, strategy="recursive")

#     cv = SingleWindowSplitter(window_length=len(y_train),fh=fh)

#     # Criar o ForecastingGridSearchCV
#     optuna_search = ForecastingOptunaSearchCV(
#         forecaster=forecaster,
#         param_grid=param_grid,
#         cv=cv,
#         n_evals=10,
#     )

    
#     #Treinando modelo
#     optuna_search.fit(y=y_train,X=X_train)

#     # Previsão
#     y_pred = optuna_search.predict(fh=fh, X=X_test)

#     # y_pred_04 = y_pred.loc['2024-11-04']

#     # # Análise de erros
#     # errors = abs(y_test['Transactioned'].values - y_pred.values)
#     # mae = errors.mean()
#     # print(f"Erro Médio Absoluto (MAE) para Seller ID {seller_id}: {mae}")
    
#     # # Plot das séries
#     # plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])
#     # plt.title(f"Previsão para Seller ID: {seller_id}")
#     # plt.show()
    
#     # Armazenar os resultados

#     ops = optuna_search.best_params_
#     # print(gs)

#     seller = {
#         "Seller ID": seller_id,
#         "Date": y_pred.index.values[0],  # Extract the single date value
#         "Score": y_pred['Transactioned'].values[0],  # Extract the single score value
#         "colsample_bytree": ops['colsample_bytree'],
#         "learning_rate": ops['learning_rate'],
#         "max_depth": ops['max_depth'],
#         "n_estimators": ops['n_estimators'],
#         "subsample": ops['subsample']
#     }

#     return seller


# # Lista de Seller IDs
# seller_ids = model_df['Seller ID'].unique()

# # Resultados para armazenar os erros por vendedor
# results = []

# for seller_id in tqdm(seller_ids, total=len(seller_ids), desc="Processando Sellers"):
#     result = process_seller_optuna(seller_id)
#     results.append(result)

# # Criar DataFrame com os resultados
# results_df = pd.DataFrame(results)
# # results_df

73 minutos

# Modelo padrão

In [899]:
def process_seller(seller_id):
    #print(f"Processando Seller ID: {seller_id}")
    
    # Filtrar os dados para o Seller ID atual
    time_serie = model_df[model_df['Seller ID'] == seller_id]
    time_serie = time_serie.drop(columns=['Seller ID','Close','Price','CBOT','Percentage Change dolar','Last Transaction','Day of Week'])
    
    # Preparação da série temporal
    time_serie.set_index('Date', inplace=True)
    target = time_serie.pop('Transactioned')
    y = pd.DataFrame(data=target, columns=['Transactioned'])
    time_serie = time_serie.shift(1)
    
    # Remover valores fora do intervalo de datas desejado
    date_to_pred = pd.to_datetime('2024-11-05')
    first_date = pd.to_datetime('2024-01-30')
    mask = (time_serie.index > first_date) & (time_serie.index < date_to_pred)
    time_serie = time_serie.loc[mask]
    y = y.loc[mask]

    # Dividir os dados em treino e teste
    y_train, y_test, X_train, X_test = temporal_train_test_split(y, time_serie, test_size=1)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_train = pd.DataFrame(data=X_train_scaled, columns=X_train.columns, index=X_train.index)
    X_test = pd.DataFrame(data=X_test_scaled, columns=X_test.columns, index=X_test.index)
    
    # Configuração do Forecasting Horizon
    fh = ForecastingHorizon([1], is_relative=True)
    
    # Calculando scale_pos_weight
    n_pos = len(y_train[y_train['Transactioned'] == 1])
    n_neg = len(y_train[y_train['Transactioned'] == 0])
    scale_pos_weight = n_neg / n_pos if n_pos > 0 else 1

    xgb_model = XGBRegressor(
        # max_depth=6,        # Moderate tree depth to prevent overfitting
        # gamma=5,            # Restricts splits to meaningful ones
        # subsample=0.8,      # Uses a subset of data for diversity
        # learning_rate=0.1,  # Smaller learning steps for stability
        # n_estimators=100,   # Number of trees
        random_state=42,
        #scale_pos_weight=scale_pos_weight,
    )
    
    # Criar o modelo
    forecaster = make_reduction(xgb_model, window_length=7, strategy="recursive")

    #Treinando modelo
    forecaster.fit(y=y_train, X=X_train)
    
    # Previsão
    y_pred = forecaster.predict(fh=fh, X=X_test)
    # y_pred_04 = y_pred.loc['2024-11-04']

    # Análise de erros
    errors = abs(y_test['Transactioned'].values - y_pred.values)
    mae = errors.mean()
    #print(f"Erro Médio Absoluto (MAE) para Seller ID {seller_id}: {mae}")
    
    # # Plot das séries
    # plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])
    # plt.title(f"Previsão para Seller ID: {seller_id}")
    # plt.show()
    
    # Armazenar os resultados
    return{
        "Seller ID": seller_id,
        "Date": y_pred.index.values[0],  # Extract the single date value
        "Score": y_pred['Transactioned'].values[0],  # Extract the single score value
        "MAE": mae
    }

# Lista de Seller IDs
seller_ids = model_df['Seller ID'].unique()

# Resultados para armazenar os erros por vendedor
results = []

for seller_id in tqdm(seller_ids, total=len(seller_ids), desc="Processando Sellers"):
    result = process_seller(seller_id)
    results.append(result)

# Criar DataFrame com os resultados
results_df = pd.DataFrame(results)
# results_df

Processando Sellers: 100%|██████████| 726/726 [00:38<00:00, 18.98it/s]


# Calculos

In [900]:
results_df

,Seller ID,Date,Score,MAE
0,100000060,2024-11-04,0.131399,0.868601
1,100002332,2024-11-04,0.000019,0.999981
2,100002035,2024-11-04,0.000022,0.999978
3,100001835,2024-11-04,-0.005866,1.005866
4,100001525,2024-11-04,-0.020051,1.020051
...,...,...,...,...
721,100000911,2024-11-04,0.000024,0.000024
722,100000993,2024-11-04,0.378575,0.378575
723,100001006,2024-11-04,0.000027,0.000027
724,100000559,2024-11-04,-0.000021,0.000021


In [901]:
media = results_df['Score'].mean()
media

0.04087213499904757

In [902]:
users_that_made_transaction_on_target_date_04 = [
    100000060,
    100000141,
    100000192,
    100000310,
    100000348,
    100001098,
    100001362,
    100001525,
    100002035,
    100002271,
    100002332,
    100002339,
    100002402,
    100002426,
]

In [903]:
users_that_made_transaction_on_target_date_05 = [
    100000180,
    100000037,
    100001254,
    10000088,
    100001203,
    100001098,
    100001599,
    100001885,
    100000608,
    100000151,
    100000638,
]

In [904]:
first_200_predictions = results_df.sort_values(
    ascending=False, by="Score"
).head(200)


first_200_predictions = first_200_predictions[
    first_200_predictions["Seller ID"].isin(users_that_made_transaction_on_target_date_04)
]

In [905]:
first_200_predictions

,Seller ID,Date,Score,MAE
5,100000192,2024-11-04,0.207696,0.792304
7,100000348,2024-11-04,0.153443,0.846557
0,100000060,2024-11-04,0.131399,0.868601
9,100001098,2024-11-04,0.091708,0.908292


In [906]:
#first_200_predictions.to_excel('C:/workspace/Desafio Grão Direto IA/Data/200_preds_opscv.xlsx',index=False)

# Coisas a fazer

- Conferir no notebook
- PCA |
- GRIDSEARCH V
- OPTUNASEARCH V
- CONCURRENT FUTURES V
- Hierarchical Clustering |

## Hierarchical Clustering

Tomar cuidado com features categoricas, até mesmo transformadas em int

In [907]:
# from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
# import matplotlib.pyplot as plt

# model_cl = model_df.drop(columns=['Seller ID', 'Transactioned'])
# model_cl = model_cl.set_index('Date')
# scaler = StandardScaler()
# model_clp = scaler.fit_transform(model_cl)
# model_clp = pd.DataFrame(model_clp, columns=model_cl.columns)
# model_clp = model_clp.transpose()

# linked = linkage(model_clp, method='ward', metric='euclidean')

# df_linked = pd.DataFrame(linked, columns=['c1', 'c2', 'distance', 'size'])
# df_linked[['c1', 'c2', 'size']] = df_linked[['c1', 'c2', 'size']].astype('Int64')

# plt.figure(figsize=(15,5))

# dendrogram(linked, orientation='top', labels=model_clp.index, distance_sort='descending', show_leaf_counts=True)
# plt.xlabel('Features')
# plt.ylabel('Wards')
# plt.show()